In [213]:

from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
import pandas as pd
import numpy as np
import pickle
import os

INPUT_DIVIDER = 0.1
START_AMOUNT = 100

fileName = 'datasets/Arbitrum_06.09.2024-06.09.2025_historical_data_coinmarketcap.csv'
delimeter = ';'
df = pd.read_csv(fileName, delimiter=delimeter, index_col='Date')
df = df.dropna()
df.sort_index()


,TimeClose,TimeHigh,TimeLow,name,Open,High,Low,Close,Volume,MarketCap,Timestamp
Date,,,,,,,,,,,
2024-09-07T00:00:00.000Z,2024-09-07T23:59:59.999Z,2024-09-07T15:31:00.000Z,2024-09-07T00:02:00.000Z,2781,0.480664,0.504619,0.480510,0.500949,1.133334e+08,1.751384e+09,2024-09-07T23:59:59.999Z
2024-09-08T00:00:00.000Z,2024-09-08T23:59:59.999Z,2024-09-08T22:31:00.000Z,2024-09-08T16:17:00.000Z,2781,0.500949,0.515120,0.497367,0.508544,1.052992e+08,1.777934e+09,2024-09-08T23:59:59.999Z
2024-09-09T00:00:00.000Z,2024-09-09T23:59:59.999Z,2024-09-09T21:21:00.000Z,2024-09-09T05:55:00.000Z,2781,0.508544,0.530742,0.506205,0.523956,1.510089e+08,1.831808e+09,2024-09-09T23:59:59.999Z
2024-09-10T00:00:00.000Z,2024-09-10T23:59:59.999Z,2024-09-10T17:13:00.000Z,2024-09-10T04:41:00.000Z,2781,0.523953,0.540089,0.516448,0.534019,1.402555e+08,1.866998e+09,2024-09-10T23:59:59.999Z
2024-09-11T00:00:00.000Z,2024-09-11T23:59:59.999Z,2024-09-11T00:00:00.000Z,2024-09-11T14:02:00.000Z,2781,0.534018,0.534018,0.502516,0.514878,1.626492e+08,1.800081e+09,2024-09-11T23:59:59.999Z
...,...,...,...,...,...,...,...,...,...,...,...
2025-09-01T00:00:00.000Z,2025-09-01T23:59:59.999Z,2025-09-01T09:15:00.000Z,2025-09-01T22:46:00.000Z,2781,0.496627,0.513153,0.468673,0.476273,4.600076e+08,2.522376e+09,2025-09-01T23:59:59.999Z
2025-09-02T00:00:00.000Z,2025-09-02T23:59:59.999Z,2025-09-02T23:17:00.000Z,2025-09-02T00:58:00.000Z,2781,0.476299,0.504673,0.475583,0.502191,3.904010e+08,2.659495e+09,2025-09-02T23:59:59.999Z
2025-09-03T00:00:00.000Z,2025-09-03T23:59:59.999Z,2025-09-03T15:11:00.000Z,2025-09-03T09:16:00.000Z,2781,0.502191,0.518888,0.493307,0.510237,3.216309e+08,2.702101e+09,2025-09-03T23:59:59.999Z


In [214]:

df.describe()

,name,Open,High,Low,Close,Volume,MarketCap
count,364.0,364.000000,364.000000,364.000000,364.000000,3.640000e+02,3.640000e+02
mean,2781.0,0.518379,0.539548,0.497009,0.518368,3.488015e+08,2.269759e+09
std,0.0,0.194612,0.204091,0.184153,0.194392,2.798516e+08,7.478449e+08
min,2781.0,0.264124,0.279977,0.245019,0.264124,5.653923e+07,1.219704e+09
25%,2781.0,0.368676,0.385883,0.357388,0.368676,1.602526e+08,1.743755e+09
50%,2781.0,0.477691,0.497759,0.461132,0.477704,2.498351e+08,2.044204e+09
75%,2781.0,0.601517,0.622279,0.575543,0.601517,4.239054e+08,2.633601e+09
max,2781.0,1.180391,1.237897,1.153918,1.175861,1.557057e+09,4.817927e+09


In [215]:

df['normal-high'] = (df['High'] -  df['High'].min()) / (df['High'].max()  - df['High'].min())
df['normal-low'] = (df['Low'] -  df['Low'].min()) / (df['Low'].max()  - df['Low'].min())
df['normal-open'] = (df['Open'] -  df['Open'].min()) / (df['Open'].max()  - df['Open'].min())
df['normal-close'] = (df['Close'] -  df['Close'].min()) / (df['Close'].max()  - df['Close'].min())

df['open-close']  = df['normal-open'] - df['normal-close']
df['low-high']  = df['normal-low'] - df['normal-high']

df['target'] = np.where(df['Close'].shift(-1) > df['Close'], 1, 0) 

featureColumns = ['open-close', 'low-high']

features = df[featureColumns]
target = df['target']

df.head(2)

,TimeClose,TimeHigh,TimeLow,name,Open,High,Low,Close,Volume,MarketCap,Timestamp,normal-high,normal-low,normal-open,normal-close,open-close,low-high,target
Date,,,,,,,,,,,,,,,,,,
2025-09-05T00:00:00.000Z,2025-09-05T23:59:59.999Z,2025-09-05T13:08:00.000Z,2025-09-05T00:01:00.000Z,2781,0.482672,0.510627,0.482522,0.494601,4.208403e+08,2.619299e+09,2025-09-05T23:59:59.999Z,0.240782,0.261309,0.238520,0.252789,-0.014269,0.020527,0
2025-09-04T00:00:00.000Z,2025-09-04T23:59:59.999Z,2025-09-04T00:37:00.000Z,2025-09-04T19:50:00.000Z,2781,0.510237,0.512834,0.479178,0.482675,2.975303e+08,2.556126e+09,2025-09-04T23:59:59.999Z,0.243086,0.257630,0.268603,0.239708,0.028895,0.014544,1


In [216]:


X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.1, random_state=34)

model = XGBClassifier(n_estimators=1, learning_rate=0.05, max_depth=3000, random_state=5)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy xboost: {accuracy:.4f}")

Accuracy xboost: 1.0000


In [217]:

# os.path.dirname(__file__) #<-- absolute dir the script is in
script_dir = os.path.join(os.path.dirname('__file__')) 


rel_path = 'models/boost_classifier-' + fileName + '.pkl'
filename = os.path.join(script_dir, rel_path)

with open(filename, 'wb') as file:
    pickle.dump(model, file)

# Load the model
with open(filename, 'rb') as file:
    model = pickle.load(file)

In [218]:
def calculateDollars(dollarsInput, inputPart):
      dlrsAmount = dollarsInput
      cryptoAmount = 0
      investAmount = inputPart * dlrsAmount

      for index, row in df[:].iterrows():
            predicted = model.predict([row[featureColumns]])[:][0]
            closePrice = row['Close']
            if predicted == 1 and dlrsAmount > investAmount and cryptoAmount == 0:
                  cryptoAmount = investAmount / closePrice
                  dlrsAmount = dlrsAmount - investAmount
            if predicted == 0 and cryptoAmount > 0:
                  dlrsAmount = dlrsAmount + cryptoAmount * closePrice
                  cryptoAmount = 0
      
      return (dlrsAmount, cryptoAmount)


def calculateSum(predicted, dlrsAmount, cryptoAmount, closePrice, investedPart):
      pass


In [219]:

dlrsAmount, cryptoAmount = calculateDollars(1000, 0.9)
print('dlrsAmount', dlrsAmount)
print('cryptoAmount', cryptoAmount)





dlrsAmount 8396.42704963684
cryptoAmount 0
